<a href="https://colab.research.google.com/github/adtitovich/numpy-38/blob/main/m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Домашнее задание «Функции и работа с данными»***

**Задание 1**

Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:
*   оценка 2 и ниже — низкий рейтинг;
*   оценка 4 и ниже — средний рейтинг;
*   оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.


In [1]:
import pandas as pd

from google.colab import drive

# монтируем google-диск для чтения ratings.csv
drive.mount('/content/gdrive') 
data = pd.read_csv('/content/gdrive/My Drive/ratings.csv')

#отключаем google-диск
drive.flush_and_unmount() 


def set_rating(rating):
  """
  классифицирует фильмы
  """
  if rating <= 2:
    return 'низкий рейтинг'
  if rating <= 4:
    return 'средний рейтинг'
  return 'высокий рейтинг'

data['class'] = data['rating'].apply(set_rating)

data.head()




Mounted at /content/gdrive


,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,средний рейтинг
1,1,1029,3.0,1260759179,средний рейтинг
2,1,1061,3.0,1260759182,средний рейтинг
3,1,1129,2.0,1260759185,низкий рейтинг
4,1,1172,4.0,1260759205,средний рейтинг


**Задание 2**

Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.


In [2]:
import pandas as pd
import re
from google.colab import drive

# монтируем google-диск для чтения keywords.csv
drive.mount('/content/gdrive') 
data = pd.read_csv('/content/gdrive/My Drive/keywords.csv')

#отключаем google-диск
drive.flush_and_unmount() 


geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
  }

def set_region(request):
  """
  Проверка request определенному региону
  """
  #поиск ключа по значению. Ищем наличие элемента списка в строке
  for key, value in geo_data.items():
    if re.findall(r'|'.join(value), request.lower()):
      return key
  return "undefined"

# добавляем новый столбец
data['region'] = data['keyword'].apply(set_region)

data[data['region'] != 'undefined' ].head()

Mounted at /content/gdrive


,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток


**Задание 3**

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:
1.   В переменную years запишите список из всех годов с 1950 по 2010 года.
2.   Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:
*   для каждой строки пройдите по всем годам списка years;
*   если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
*   если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.
3.   Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [37]:
import pandas as pd
import re
from google.colab import drive

# монтируем google-диск для чтения .csv
drive.mount('/content/gdrive') 
movies = pd.read_csv('/content/gdrive/My Drive/movies.csv')
ratings = pd.read_csv('/content/gdrive/My Drive/ratings.csv')

#отключаем google-диск
drive.flush_and_unmount() 

#список годов
years = [str(x) for x in list(range(1950, 2011))]


def production_year(title):
  """
   из названия фильма выставляет год выпуска
  """
  result = re.findall(r'|'.join(years), title)
  if result:
    return result[-1]
  else:
    return '1900'

# добавляем новый столбец
movies['year'] = movies['title'].apply(production_year)

# объединяем dataframes
data = pd.merge(movies, ratings, on='movieId', how='inner')

# средний рейтинг с сортировкой по убыванию
data.groupby('year').mean()[['rating']].sort_values('rating', ascending=False).head(51)



Mounted at /content/gdrive


,rating
year,
1957,4.014241
1972,4.011136
1952,4.000000
1954,3.994220
1951,3.983539
1974,3.978704
1962,3.952446
1950,3.915254
1977,3.905786
